<a href="https://colab.research.google.com/github/dTenebrae/nlp/blob/main/lesson9/hw9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import datetime
import re

from tensorflow.python.client import device_lib
print(f"Tensorflow version: {tf.__version__}")
dev = re.findall(r"name:\W(.+),", device_lib.list_local_devices()[1].physical_device_desc)[0].split(",")[0]
print(f"GPU: {dev}")

Tensorflow version: 2.7.0
GPU: Tesla P100-PCIE-16GB


In [2]:
with open("/content/drive/MyDrive/data/iliada.txt") as f:
    file_list = f.readlines()

In [3]:
len(file_list)

44569

In [4]:
trimmed_text = []
for line in file_list:
    comp_line = re.findall(r"[\d\s]+", line)[0]
    if (line != "\n") and (line != comp_line):
        line = line.replace("\t", " ")
        line = re.sub(r"\[\d+\]", "", line)
        trimmed_text.append(line)
len(trimmed_text)

16845

In [5]:
trimmed_text[:20]

['Гомер\n',
 'Илиада\n',
 'Песнь первая\n',
 'Язва. Гнев\n',
 'Гнев, богиня,  воспой Ахиллеса,  Пелеева сына, Грозный, который ахеянам тысячи бедствий соделал:\n',
 'Многие души могучие славных героев низринул\n',
 'В мрачный Аид и самих распростер их в корысть плотоядным\n',
 'Птицам окрестным и псам (совершалася Зевсова воля), —\n',
 'С оного дня, как, воздвигшие спор, воспылали враждою\n',
 'Пастырь народов Атрид и герой Ахиллес благородный.\n',
 'Кто ж от богов бессмертных подвиг их к враждебному спору?\n',
 'Сын громовержца и Леты – Феб,  царем прогневленный,\n',
 'Язву на воинство злую навел; погибали народы\n',
 'В казнь, что Атрид обесчестил жреца непорочного Хриса.\n',
 'Старец, он приходил к кораблям быстролетным ахейским\n',
 'Пленную дочь искупить и, принесши бесчисленный выкуп\n',
 'И держа в руках, на жезле золотом, Аполлонов\n',
 'Красный венец,  умолял убедительно всех он ахеян, Паче ж Атридов могучих, строителей рати ахейской:\n',
 '“Чада Атрея и пышнопоножные мужи ахе

In [6]:
result = 0
for line in trimmed_text:
    result += len(line.split())
print(f"Number of tokens: {result}")

Number of tokens: 133448


In [7]:
joined_text = "".join(trimmed_text)

In [8]:
print(joined_text[:500])

Гомер
Илиада
Песнь первая
Язва. Гнев
Гнев, богиня,  воспой Ахиллеса,  Пелеева сына, Грозный, который ахеянам тысячи бедствий соделал:
Многие души могучие славных героев низринул
В мрачный Аид и самих распростер их в корысть плотоядным
Птицам окрестным и псам (совершалася Зевсова воля), —
С оного дня, как, воздвигшие спор, воспылали враждою
Пастырь народов Атрид и герой Ахиллес благородный.
Кто ж от богов бессмертных подвиг их к враждебному спору?
Сын громовержца и Леты – Феб,  царем прогневленны


In [9]:
vocab = sorted(set(joined_text))
print(f"{len(vocab)} unique characters")

106 unique characters


In [10]:
char2int = {c:i for i, c in enumerate(vocab)}
int2char = np.array(vocab)
for char,_ in zip(char2int, range(len(vocab))):
    print("{:4s}: {:3d},".format(repr(char), char2int[char]))

'\n':   0,
' ' :   1,
'!' :   2,
'(' :   3,
')' :   4,
',' :   5,
'-' :   6,
'.' :   7,
'0' :   8,
'1' :   9,
'2' :  10,
'3' :  11,
'4' :  12,
'5' :  13,
'6' :  14,
'7' :  15,
'8' :  16,
'9' :  17,
':' :  18,
';' :  19,
'?' :  20,
'B' :  21,
'D' :  22,
'E' :  23,
'O' :  24,
'T' :  25,
'V' :  26,
'a' :  27,
'c' :  28,
'e' :  29,
'i' :  30,
'l' :  31,
'm' :  32,
'n' :  33,
'o' :  34,
's' :  35,
't' :  36,
'u' :  37,
'«' :  38,
'»' :  39,
'А' :  40,
'Б' :  41,
'В' :  42,
'Г' :  43,
'Д' :  44,
'Е' :  45,
'Ж' :  46,
'З' :  47,
'И' :  48,
'К' :  49,
'Л' :  50,
'М' :  51,
'Н' :  52,
'О' :  53,
'П' :  54,
'Р' :  55,
'С' :  56,
'Т' :  57,
'У' :  58,
'Ф' :  59,
'Х' :  60,
'Ц' :  61,
'Ч' :  62,
'Ш' :  63,
'Щ' :  64,
'Э' :  65,
'Ю' :  66,
'Я' :  67,
'а' :  68,
'б' :  69,
'в' :  70,
'г' :  71,
'д' :  72,
'е' :  73,
'ж' :  74,
'з' :  75,
'и' :  76,
'й' :  77,
'к' :  78,
'л' :  79,
'м' :  80,
'н' :  81,
'о' :  82,
'п' :  83,
'р' :  84,
'с' :  85,
'т' :  86,
'у' :  87,
'ф' :  88,
'х' :  89,
'ц' :  90,

In [11]:
text_as_int = np.array([char2int[ch] for ch in joined_text], dtype=np.int32)
print("{}\n mapped to integers:\n {}".format(repr(joined_text[:100]), text_as_int[:100]))

'Гомер\nИлиада\nПеснь первая\nЯзва. Гнев\nГнев, богиня,  воспой Ахиллеса,  Пелеева сына, Грозный, который'
 mapped to integers:
 [43 82 80 73 84  0 48 79 76 68 72 68  0 54 73 85 81 96  1 83 73 84 70 68
 99  0 67 75 70 68  7  1 43 81 73 70  0 43 81 73 70  5  1 69 82 71 76 81
 99  5  1  1 70 82 85 83 82 77  1 40 89 76 79 79 73 85 68  5  1  1 54 73
 79 73 73 70 68  1 85 95 81 68  5  1 43 84 82 75 81 95 77  5  1 78 82 86
 82 84 95 77]


In [12]:
tr_text = text_as_int[:700_000] 
val_text = text_as_int[700_000:] 
print(text_as_int.shape, tr_text.shape, val_text.shape)

(868003,) (700000,) (168003,)


In [13]:
batch_size = 64
buffer_size = 10000
embedding_dim = 256
epochs = 50
seq_length = 200
examples_per_epoch = len(joined_text)//seq_length
rnn_units = 1024
vocab_size = len(vocab)

In [14]:
tr_char_dataset = tf.data.Dataset.from_tensor_slices(tr_text)
val_char_dataset = tf.data.Dataset.from_tensor_slices(val_text)
tr_sequences = tr_char_dataset.batch(seq_length+1, drop_remainder=True)
val_sequences = val_char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text
    
tr_dataset = tr_sequences.map(split_input_target).shuffle(buffer_size).batch(batch_size, drop_remainder=True)
val_dataset = val_sequences.map(split_input_target).shuffle(buffer_size).batch(batch_size, drop_remainder=True)
print(tr_dataset, val_dataset)

<BatchDataset shapes: ((64, 200), (64, 200)), types: (tf.int32, tf.int32)> <BatchDataset shapes: ((64, 200), (64, 200)), types: (tf.int32, tf.int32)>


In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
                                 tf.keras.layers.Embedding(vocab_size, 
                                                           embedding_dim, 
                                                           batch_input_shape=[batch_size, None]),
                                 tf.keras.layers.Dropout(0.2),
                                 tf.keras.layers.LSTM(rnn_units, 
                                                      return_sequences=True, 
                                                      stateful=True, 
                                                      recurrent_initializer="glorot_uniform"),
                                 tf.keras.layers.Dropout(0.2), 
                                 tf.keras.layers.LSTM(rnn_units, 
                                                      return_sequences=True, 
                                                      stateful=True, 
                                                      recurrent_initializer="glorot_uniform"),
                                 tf.keras.layers.Dropout(0.2),
                                 tf.keras.layers.Dense(vocab_size)
                                  ])
    return model

In [16]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=batch_size
    )

In [17]:
model.summary()
for input_example_batch, target_example_batch in tr_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print(f"Loss: {example_batch_loss.numpy().mean()}")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           27136     
                                                                 
 dropout (Dropout)           (64, None, 256)           0         
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dropout_1 (Dropout)         (64, None, 1024)          0         
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dropout_2 (Dropout)         (64, None, 1024)          0         
                                                                 
 dense (Dense)               (64, None, 106)           1

In [18]:
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss)
patience = 10
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

In [19]:
checkpoint_dir = "./checkpoints"+ datetime.datetime.now().strftime("_%Y.%m.%d-%H:%M:%S")
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [20]:
history = model.fit(tr_dataset, 
                    epochs=epochs, 
                    callbacks=[early_stop, checkpoint_callback] , 
                    validation_data=val_dataset)
print(f"Training stopped as there was no improvement after {patience} epochs")

Epoch 1/50
54/54 [==============================] - 19s 269ms/step - loss: 3.4352 - val_loss: 3.2678
Epoch 2/50
54/54 [==============================] - 14s 251ms/step - loss: 2.9747 - val_loss: 2.9469
Epoch 3/50
54/54 [==============================] - 15s 253ms/step - loss: 2.6218 - val_loss: 2.7975
Epoch 4/50
54/54 [==============================] - 15s 252ms/step - loss: 2.4227 - val_loss: 2.6518
Epoch 5/50
54/54 [==============================] - 15s 253ms/step - loss: 2.2514 - val_loss: 2.5169
Epoch 6/50
54/54 [==============================] - 15s 252ms/step - loss: 2.0762 - val_loss: 2.4126
Epoch 7/50
54/54 [==============================] - 15s 252ms/step - loss: 1.9197 - val_loss: 2.3152
Epoch 8/50
54/54 [==============================] - 14s 251ms/step - loss: 1.7926 - val_loss: 2.2268
Epoch 9/50
54/54 [==============================] - 15s 252ms/step - loss: 1.6923 - val_loss: 2.1722
Epoch 10/50
54/54 [==============================] - 14s 251ms/step - loss: 1.6102 - val_lo

In [21]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir)) 
model.build(tf.TensorShape([1, None]))
def generate_text(model, start_string):
    print('Generating with seed: "' + start_string + '"')
  
    num_generate = 1000
    input_eval = [char2int[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temperature = 1.0
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(int2char[predicted_id])
    return (start_string + ''.join(text_generated))

print(generate_text(model, start_string="В мрачный Аид и самих"))

Generating with seed: "В мрачный Аид и самих"
В мрачный Аид и самихов и духом предстал он:
“Храбрый если обоих ты оставь совершенно издебренновно?
Если ж ты, уже в сражению! Дордую мчатся
Вслад пред тобой поражают Менета и смертных супруга.
Если б эни бы справедли глабокий малает, Тера, Анфимеха Аргоса грома, смира в челоезате, —
Я на тебя улылого, возвести мне я, повелимом
Я в илион по
пошлюд и кразешь их деня изло многих.
Рек Лиимоп, и сама обращенно вершине, —
Так произнес – на другое что дивную славу я буду!
Зовчу на Идошей медь, сердца то жилость вещаешь?
Так он Тевфон, поразила в противенной руки Арея.
Тою порою прили, за полу волна Троа слушаться!
Истинно сына и разра, помодит и могущество духа,
Мужа величесь цвего, как своед корабли их давесца
Коней черного не будет великая смертного сына.
Тов, лишь Афина сыны виношил, но презренные дени
В вые от Зевса героя, чудос на свирепстве его злопотучное рато;
Гактора матерь любезная: медь ты сыны беспрестанно
Паден тельцов ни бесполнить